# Import Library & Load Konfigurasi

In [ ]:
import os
import json
import vertexai
from typing import List, Dict, Any, Union, TypedDict, Tuple

# Import LangChain & Vertex AI Components
from langchain.embeddings.base import Embeddings
from langchain_google_vertexai import ChatVertexAI #deprecated
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_pinecone import PineconeVectorStore
from vertexai.vision_models import MultiModalEmbeddingModel, Image 
from pinecone import Pinecone,ServerlessSpec 
from dotenv import load_dotenv
# from langchain_google_vertexai import VertexAIEmbeddings
from langchain_core.documents import Document
from google.oauth2 import service_account
import base64
from IPython.display import display, Markdown
import numpy as np
from collections import defaultdict
import operator

Selanjutnya, kita akan coba memuat konfigurasi untuk GCP dan pinecone.

In [2]:
# Load variabel dari .env
load_dotenv()
GOOGLE_APPLICATION_CREDENTIALS = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")

PROJECT_ID = os.environ.get("PROJECT_ID")
LOCATION = os.environ.get("REGION")
environment = "us-west1-gcp"  # sesuaikan region Anda
index_name = "products-index"
print(GOOGLE_APPLICATION_CREDENTIALS)
print(PROJECT_ID)
print(LOCATION)

qwiklabs-gcp-03-18343d304453-bc85198ddc9d.json
qwiklabs-gcp-04-245b6c77016f
us-west1


In [4]:
#inisialisasi Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# inisialisasi Klien pinecone
pc= Pinecone(api_key=os.environ["PINECONE_API_KEY"],
              environment=environment)

pc.list_indexes().names()

['laptop-recommendation',
 'laptops-index-v1',
 'langchain-demo',
 'demo-llamaindex-langchain',
 'products-index']

In [ ]:
# pc.delete_index(index_name)

# Membuat Index Pinecone

In [65]:
# buat index baru
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=128,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws", region="us-east-1"
            
        )
    )


Pembuatan index tersebut kita memnggunakan `dimensi=128`, dimensi ini harus sesuai dengan dimensi dari embedding yang akan kita buat.

In [4]:
index = pc.Index(index_name)

In [6]:
Embeddings

langchain_core.embeddings.embeddings.Embeddings

In [5]:
class VertexAIMultiModalEmbeddings:
    """
    Class ini digunakan untuk menghasilkan embedding dari:
    - Gambar
    - Teks
    - Kombinasi gambar + teks (multimodal)
    """

    def __init__(
        self,
        model_name: str = "multimodalembedding@001",
        dimension: int = 128,
    ):
        """
        Inisialisasi model Vertex AI MultiModal Embedding.

        Args:
            model_name (str):
                Nama model Vertex AI multimodal embedding.
                Default: "multimodalembedding@001"

            dimension (int):
                Dimensi vektor embedding yang dihasilkan.
                Contoh: 128, 256, 512
        """
        self.model = MultiModalEmbeddingModel.from_pretrained(model_name)
        self.dimension = dimension

    def embed_image_and_text(
        self,
        image_path: str,
        contextual_text: str = "",
    ) -> Tuple[List[float], List[float]]:
        """
        Menghasilkan embedding dari gambar dan teks secara bersamaan.

        Method ini mengembalikan dua vektor embedding yang terpisah:
        - Image embedding
        - Text embedding

        Args:
            image_path (str):
                Path ke file gambar lokal.

            contextual_text (str, optional):
                Teks konteks yang berkaitan dengan gambar.
                Default: string kosong.

        Returns:
            Tuple[List[float], List[float]]:
                - image_embedding: Vektor embedding untuk gambar
                - text_embedding: Vektor embedding untuk teks
        """
        if image_path == "":
            image=None
        else:
            image = Image.load_from_file(image_path)

        result = self.model.get_embeddings(
            image=image,
            contextual_text=contextual_text,
            dimension=self.dimension,
        )

        return result.image_embedding, result.text_embedding

Selanjutnya, kita akan menguji coba dnegan beberapa daftar produk furniture.

In [68]:
products = [
    {
        "id": "prod_001",
        "name": "Kursi Kantor Ergonomis",
        "price": 1500000,
        "category": "Furniture",
        "description": "Kursi kantor dengan sandaran punggung ergonomis, tinggi dapat diatur, dilengkapi dengan armrest yang nyaman, roda putar 360°, cocok untuk bekerja di meja sepanjang hari tanpa pegal.",
        "image_path": "gs://latihan-my-ecommerce-images1/images/kursi_kantor.jpg"
    },
    {
        "id": "prod_002",
        "name": "Kursi Gaming Premium",
        "price": 2000000,
        "category": "Furniture",
        "description": "Kursi gaming ergonomis bergaya racing ini, sangat cocok untuk gamer profesional dan streamer yang membutuhkan kenyamanan intens, dirancang dengan material pelapis Kulit Sintetis (PU Leather) yang mudah dibersihkan dan rangka internal Baja (Steel Frame) yang kokoh, serta tampil mencolok dengan kombinasi warna Merah dan Hitam yang agresif, dilengkapi dengan fitur unggulan seperti bantal leher dan pinggang serta mekanisme reclining untuk penyesuaian postur optimal.",
        "image_path": "gs://latihan-my-ecommerce-images1/images/kursi_gaming.jpg"
    },
    {
        "id": "prod_003",
        "name": "Kursi Santai Minimalis",
        "price": 100000,
        "category": "Furniture",
        "description": "Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.",
        "image_path": "gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg"
    },
    {
        "id": "prod_004",
        "name": "Meja Kayu Minimalis",
        "price": 150000,
        "category": "Furniture",
        "description": "Meja kayu solid ukuran 120x60cm, finishing natural, kaki kayu kokoh, cocok untuk ruang kerja atau belajar, mudah dibersihkan dan tahan lama.",
        "image_path": "gs://latihan-my-ecommerce-images1/images/meja_kayu.jpg"
    },
    {
        "id": "prod_005",
        "name": "Lampu Meja LED",
        "price": 80000,
        "category": "Lighting",
        "description": "Lampu meja LED dengan intensitas cahaya dapat diatur, desain modern minimalis, hemat energi, cocok untuk meja belajar, bekerja, atau membaca di malam hari.",
        "image_path": "gs://latihan-my-ecommerce-images1/images/lampu_meja.jpg"
    },
     {
        "id": "prod_006",
        "name": "Rexus Gaming Chair Kursi RGC-R60 / R60 - Kuning",
        "price": 2100000,
        "category": "Furniture",
        "description": "Rexus R60 dilengkapi dengan sandaran yang bisa dimiringkan (reclining) hingga 125 derajat. Sudut kemiringan tersebut sudah dipertimbangkan faktor keamanannya, bahkan saat kaki berada di atas sandaran kaki. Rexus R60 menampilkan fitur revolusioner pada sebuah kursi gaming, yaitu footrest atau sandaran kaki. Footrest tersebut terintegrasi dengan kursi dan dapat diatur posisinya sehingga mampu menopang kaki Anda secara sempurna.",
        "image_path": "gs://latihan-my-ecommerce-images1/images/kursi_rexus.jpg"
    },
    {
        "id": "prod_007",
        "name": "Kursi Santai Minimalis Kontemporer",
        "price": 250000,
        "category": "Furniture",
        "description": "Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.",
        "image_path": "gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg"
    }
]

In [ ]:
# model = MultiModalEmbeddingModel.from_pretrained("multimodalembedding@001")
# embedding_dimension = 128

Untuk melakukan embedding image dan text kita bisa gunakan `multimodalembedding@001`

In [6]:
embeddings_model = VertexAIMultiModalEmbeddings(model_name="multimodalembedding@001", dimension=128)
# vector_store = PineconeVectorStore(index=index, embedding=embeddings_model)

d:\ml_project\stock market\.venv\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [ ]:
def create_vectors_for_pinecone(products: List[Dict], embeddings_model) -> List[Dict]:
    """Menghitung dua vektor (Teks dan Gambar) dan memformatnya untuk upsert Pinecone."""
    pinecone_vectors = []
    
    for product in products:
        product_id = product["id"]
        context_text = f"Nama: {product['name']}, Harga:{product['price']} ,Kategori: {product['category']}, Deskripsi: {product['description']}"
        image_path = product["image_path"]
        
        try:
            # Kita perlu fungsi baru di model untuk mengembalikan kedua vektor (image dan text)
            image_vector, text_vector = embeddings_model.embed_image_and_text(
                image_path=image_path, 
                contextual_text=context_text
            )
              
            # Metadata umum
            metadata = {
                "product_id": product["id"],
                "name": product["name"],
                "price": product["price"],
                "category": product["category"],
                "description": product["description"],
                "image_path": product["image_path"],
                "source": "multi-modal-product-catalog"
            }
            
            # --- Entri 1: Vektor Gambar (Image) ---
            pinecone_vectors.append({
                "id": f"{product_id}-IMG", # ID unik untuk vektor gambar
                "values": image_vector,
                "metadata": {**metadata, "vector_type": "image"} # Tambahkan tipe vektor
            })
            
            # --- Entri 2: Vektor Teks (Text) ---
            pinecone_vectors.append({
                "id": f"{product_id}-TXT", # ID unik untuk vektor teks
                "values": text_vector,
                "metadata": {**metadata, "vector_type": "text"} # Tambahkan tipe vektor
            })
            
            print(f"Sukses memproses produk: {product_id} (2 vektor dibuat)")
            
        except Exception as e:
            print(f"Gagal memproses produk {product_id}: {e}")
            continue
            
    return pinecone_vectors

create_vectors_for_pinecone(products,embeddings_model)

Sukses memproses produk: prod_001 (2 vektor dibuat)
Sukses memproses produk: prod_002 (2 vektor dibuat)
Sukses memproses produk: prod_003 (2 vektor dibuat)
Sukses memproses produk: prod_004 (2 vektor dibuat)
Sukses memproses produk: prod_005 (2 vektor dibuat)
Sukses memproses produk: prod_006 (2 vektor dibuat)
Sukses memproses produk: prod_007 (2 vektor dibuat)


[{'id': 'prod_001-IMG',
  'values': [0.0478857756,
   -0.0765928328,
   -0.0609029196,
   0.0669969767,
   -0.427656621,
   -0.0176772568,
   0.00698836055,
   -0.039904952,
   -0.0445451,
   0.154176757,
   -0.107244037,
   0.0112158898,
   -0.0464473,
   0.000526373624,
   -0.0425389223,
   0.0450653806,
   -0.0338132568,
   -0.0416115411,
   -0.0378993116,
   -0.00871994626,
   0.0245756507,
   -0.00771715119,
   0.0327135101,
   0.0529604591,
   -0.105699696,
   -0.026671797,
   0.0129509829,
   0.254727244,
   0.00178044406,
   -0.0159838069,
   -0.0320251323,
   0.208395138,
   0.0527725294,
   -0.028249735,
   0.00283712381,
   -0.105347477,
   -0.0510768704,
   0.132586524,
   -0.0431229547,
   0.120922863,
   -0.04632001,
   -0.0203283411,
   0.0239749774,
   0.00800831802,
   -0.0454311371,
   -0.0121720321,
   -0.0770480186,
   0.0456415154,
   0.0898004249,
   -0.0417671874,
   -0.127160698,
   0.0619651712,
   0.0102969753,
   -0.0198363867,
   -0.00626638252,
   0.0515002

In [ ]:
def index_products_to_pinecone(products_data):
    """Fungsi utama untuk menjalankan indexing atau pembuatan idex dari vektor yang telah dibuat."""
    print("--- MEMULAI INDEXING PRODUK KE PINECONE ---")
    formatted_vectors = create_vectors_for_pinecone(products_data, embeddings_model=embeddings_model)

    if formatted_vectors:
        try:
            upsert_response = index.upsert(vectors=formatted_vectors) 
            print("\n✅ Proses Indexing ke Pinecone Selesai!")
            print(f"Vectors upserted: {upsert_response['upserted_count']}")
        except Exception as e:
            print(f"\n❌ Gagal saat Upsert ke Pinecone: {e}")
    else:
        print("\nTidak ada vektor yang dihasilkan untuk upsert.")

In [73]:
index_products_to_pinecone(products)

--- MEMULAI INDEXING PRODUK KE PINECONE ---
Sukses memproses produk: prod_001 (2 vektor dibuat)
Sukses memproses produk: prod_002 (2 vektor dibuat)
Sukses memproses produk: prod_003 (2 vektor dibuat)
Sukses memproses produk: prod_004 (2 vektor dibuat)
Sukses memproses produk: prod_005 (2 vektor dibuat)
Sukses memproses produk: prod_006 (2 vektor dibuat)
Sukses memproses produk: prod_007 (2 vektor dibuat)

✅ Proses Indexing ke Pinecone Selesai!
Vectors upserted: 14


In [74]:
# cek index
index.describe_index_stats()

{'dimension': 128,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 14}},
 'total_vector_count': 14,
 'vector_type': 'dense'}

In [ ]:
# # cek list index
# pc.list_indexes().names()

# Merancang Prompt dan Skema Rekomendasi Produk

In [ ]:
credentials = service_account.Credentials.from_service_account_file(
    GOOGLE_APPLICATION_CREDENTIALS,
    scopes=["https://www.googleapis.com/auth/cloud-platform"],
)


llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    credentials=credentials,
    project=PROJECT_ID,
    vertexai=True, 
)

# testing LLM dengan input multimodal , image dalam bentuk base64
content_blocks=[]
query_image_path = 'images/image.png'  # Ganti dengan path gambar yang sesuai
image_bytes = open(query_image_path, "rb").read()
image_base64 = base64.b64encode(image_bytes).decode("utf-8")
mime_type = "image/png"

# Jika menggunakan google storage, kita dapat gunakan fungsi 'load_from_file'
query_image_path = 'https://storage.googleapis.com/cloud-training/OCBL447/gemini-app/images/living_room.jpeg'
image = Image.load_from_file(query_image_path)
query_image_path= ''

if query_image_path:
    content_blocks.append({
        "type": "image",
        "url": query_image_path,
        # "base64": image_base64,
        # "mime_type": mime_type,
    })


query_text = "Berikan saya rekomendasi furniture kursi untuk ruang tamu berikut"
content_blocks.append(
    {
    "type": "text",
    "text": f"{query_text}."
    }
)
print(content_blocks)

system_prompt = (
"Anda adalah sistem analisis produk furniture."
"Outputnya harus berupa JSON valid dengan dua kunci: 'is_furniture' (boolean) dan 'description' (string)."
"is_furniture bernilai True jika Input berkaitan dengan furniture (meja, kursi, sofa, lemari, dll)."
"description (<1000 karakter): ringkasan gambar, kegunaan, material produk dan warna."
"Contoh output: {'is_furniture': True, 'description':'gambar tersebut merupakan....'}."
)

messages = [
    SystemMessage(content=system_prompt),
    HumanMessage(content=content_blocks)
]
response = llm.invoke(messages)
print(response.content)

d:\ml_project\stock market\.venv\Lib\site-packages\vertexai\vision_models\_vision_models.py:154: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


[{'type': 'text', 'text': 'Berikan saya rekomendasi furniture kursi untuk ruang tamu berikut.'}]
```json
{
  "is_furniture": true,
  "description": "Gambar tersebut menampilkan rekomendasi furniture kursi untuk ruang tamu. Kursi ini dirancang untuk memberikan kenyamanan dan estetika pada ruang keluarga atau ruang tamu. Umumnya terbuat dari rangka kayu solid atau metal, dengan bantalan busa empuk yang dilapisi kain berkualitas tinggi seperti beludru, linen, atau kulit sintetis. Tersedia dalam berbagai pilihan warna mulai dari netral seperti abu-abu, krem, hingga warna cerah atau bermotif untuk menyesuaikan gaya interior ruangan."
}
```


Pada kode tersebut, respon yang di dapat akan membentuk json, json ini digunakan untuk melakukan validasi apakah pertanyaan user berkaitan dengan furniture atau tidak, jika ya maka akan diberikan rekomendasi produk yang sesuai, jika tidak maka proses akan dihentikan atau tidak diberikan rekomendasi.

In [12]:
content_blocks

[{'type': 'image',
  'url': 'https://storage.googleapis.com/cloud-training/OCBL447/gemini-app/images/living_room.jpeg'},
 {'type': 'text',
  'text': 'Berikan saya rekomendasi furniture untuk ruang tamu berikut.'}]

In [13]:
response.content

'```json\n{\n  "is_furniture": true,\n  "description": "Gambar ini menampilkan ruang tamu dengan nuansa modern bohemian yang didominasi warna netral. Terdapat sofa abu-abu muda dengan bantalan empuk dan kaki kayu, cocok untuk tiga orang. Di sebelahnya ada kursi berlengan tunggal berwarna krem gading dengan bantal bulat, memberikan kenyamanan ekstra. Meja kopi bundar dengan permukaan marmer putih dan kaki kayu berfungsi sebagai pusat ruangan. Sebuah pouf rajutan bulat berwarna krem gading dapat digunakan sebagai pijakan kaki atau tempat duduk tambahan. Lampu lantai dengan tiang kayu dan kap kain krem menyediakan pencahayaan ambient. Karpet anyaman besar berwarna krem menutupi lantai kayu, menciptakan suasana hangat. Furnitur ini terbuat dari kombinasi kain, kayu, dan marmer."\n}\n```'

Karena hasil response dalam bentuk text atau string, maka kita perlu mengubahnya atau membersihkan hasil response agar menjadi format json.

In [11]:
raw = response.text.strip()
raw = raw.replace("```json", "").replace("```", "").strip()

try:
    result = json.loads(raw)
    print(result)
except Exception as e:
    print("Gagal parse JSON:", e)
    print("Raw output:", raw)

{'is_furniture': True, 'description': 'Gambar tersebut menampilkan rekomendasi furniture kursi untuk ruang tamu. Kursi ini dirancang untuk memberikan kenyamanan dan estetika pada ruang keluarga atau ruang tamu. Umumnya terbuat dari rangka kayu solid atau metal, dengan bantalan busa empuk yang dilapisi kain berkualitas tinggi seperti beludru, linen, atau kulit sintetis. Tersedia dalam berbagai pilihan warna mulai dari netral seperti abu-abu, krem, hingga warna cerah atau bermotif untuk menyesuaikan gaya interior ruangan.'}


In [12]:
query = " ".join(result.get("description", "").split())
is_furniture = result.get("is_furniture")
print(f"Query: {query}, Is Furniture: {is_furniture}")

Query: Gambar tersebut menampilkan rekomendasi furniture kursi untuk ruang tamu. Kursi ini dirancang untuk memberikan kenyamanan dan estetika pada ruang keluarga atau ruang tamu. Umumnya terbuat dari rangka kayu solid atau metal, dengan bantalan busa empuk yang dilapisi kain berkualitas tinggi seperti beludru, linen, atau kulit sintetis. Tersedia dalam berbagai pilihan warna mulai dari netral seperti abu-abu, krem, hingga warna cerah atau bermotif untuk menyesuaikan gaya interior ruangan., Is Furniture: True


In [61]:
query_image_path

'https://storage.googleapis.com/cloud-training/OCBL447/gemini-app/images/living_room.jpeg'

In [ ]:
# query_image_path='images/kursi_kayu.jpg'

In [63]:
len(query)

617

In [57]:
query

'Gambar tersebut menampilkan rekomendasi furniture kursi untuk ruang tamu. Kursi ini dirancang untuk memberikan kenyamanan dan estetika pada ruang keluarga atau ruang tamu. Umumnya terbuat dari rangka kayu solid atau metal, dengan bantalan busa empuk yang dilapisi kain berkualitas tinggi seperti beludru, linen, atau kulit sintetis. Tersedia dalam berbagai pilihan warna mulai dari netral seperti abu-abu, krem, hingga warna cerah atau bermotif untuk menyesuaikan gaya interior ruangan.'

Jika image tidak ada maka query_vector_image akan kosong dan mengakibatkan error ketika melakukan query berdasarkan image ke pinecone.

In [58]:
# Embed query menggunakan model multimodal
query_vector_image,query_vector_text=embeddings_model.embed_image_and_text(
    image_path=query_image_path,
    contextual_text=query
)


TOP_K = 3  # Jumlah hasil yang dicari dari setiap query

# --- QUERY 1: Pencarian Berbasis Teks ---
# Cari kesamaan vektor TEKS query dengan Vektor produk bertipe 'text'
text_search_results = index.query(
    vector=query_vector_text, 
    top_k=TOP_K, 
    include_metadata=True,
    # PENTING: Filter hanya Vektor Teks
    filter={"vector_type": "text"} 
)


In [59]:
text_search_results

{'matches': [{'id': 'prod_007-TXT',
              'metadata': {'category': 'Furniture',
                           'description': 'Kursi santai tunggal bergaya '
                                          'Minimalis Kontemporer yang cocok '
                                          'diletakkan di sudut baca atau '
                                          'sebagai kursi aksen di ruang tamu, '
                                          'terbuat dari Material Pelapis Kain '
                                          'Tenun berwarna Abu-abu Muda yang '
                                          'nyaman, ditopang oleh Kaki Kayu '
                                          'Solid Alami yang tirus, memberikan '
                                          'sentuhan kehangatan dan elegan.',
                           'image_path': 'gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg',
                           'name': 'Kursi Santai Minimalis Kontemporer',
                           'price': 25

In [ ]:
# --- QUERY 2: Pencarian Berbasis Gambar ---
# Cari kesamaan vektor GAMBAR query dengan Vektor produk bertipe 'image'
image_search_results = index.query(
    vector=query_vector_image, 
    top_k=TOP_K, 
    include_metadata=True,
    # PENTING: Filter hanya Vektor Gambar
    filter={"vector_type": "image"}
)


In [16]:
image_search_results

{'matches': [{'id': 'prod_006-IMG',
              'metadata': {'category': 'Furniture',
                           'description': 'Rexus R60 dilengkapi dengan '
                                          'sandaran yang bisa dimiringkan '
                                          '(reclining) hingga 125 derajat. '
                                          'Sudut kemiringan tersebut sudah '
                                          'dipertimbangkan faktor keamanannya, '
                                          'bahkan saat kaki berada di atas '
                                          'sandaran kaki. Rexus R60 '
                                          'menampilkan fitur revolusioner pada '
                                          'sebuah kursi gaming, yaitu footrest '
                                          'atau sandaran kaki. Footrest '
                                          'tersebut terintegrasi dengan kursi '
                                          'dan dapat diatur posis

In [67]:
context = []
for match in text_search_results.matches:
    product_info = (
        f"ID Produk: {match.id}, "
        f"Score: {match.score:.4f}, "
        f"Nama: {match.metadata.get('name', 'N/A')}, "
        f"Harga: {match.metadata.get('price', 'N/A')}, "
        f"Kategori: {match.metadata.get('category', 'N/A')}, "
        f"Deskripsi: {match.metadata.get('description', 'Tidak ada deskripsi')},"
        f"Image Path: {match.metadata.get('image_path', 'N/A')}"     
    )
    context.append(product_info)
    
print(context)

['ID Produk: prod_007-TXT, Score: 0.7838, Nama: Kursi Santai Minimalis Kontemporer, Harga: 250000.0, Kategori: Furniture, Deskripsi: Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.,Image Path: gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg', 'ID Produk: prod_003-TXT, Score: 0.7587, Nama: Kursi Santai Minimalis, Harga: 100000.0, Kategori: Furniture, Deskripsi: Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.,Image Path: gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg', 'ID Produk: prod_004-TXT, Score: 0.7562, Nama: Meja Kayu Minimalis, Harga: 150000.0, Kategori: Furniture, Deskripsi: Meja kayu solid ukuran

In [69]:
llm.invoke(context).usage_metadata

{'input_tokens': 370,
 'output_tokens': 2303,
 'total_tokens': 2673,
 'input_token_details': {'cache_read': 0},
 'output_token_details': {'reasoning': 1820}}

Selanjutnya, kita akan coba buat fungsi yang melakukan pengecekan terlebih dahulu terhadap input text dan image apakah ada atau tidak, kemudian membuat pencarian produk berdasarkan consine similarity dengan mengkombinasi skor text dan image, jika input dari user memberikan text dan image.

In [ ]:
# Dictionary untuk melacak skor Teks dan Gambar terpisah
# Default value untuk setiap skor adalah 0.0 (jika tidak ditemukan)
combined_modality_scores = defaultdict(lambda: {
    'text_score': 0.0,
    'image_score': 0.0,
    'metadata': None
})

def process_pinecone_results(
    results,
    combined_scores: dict,
    vector_type: str,
    min_score: float
):
    
    for match in results.matches:
        # Lewati hasil dengan skor rendah
        if match.score < min_score:
            continue

        # Ambil product_id (tanpa suffix -text / -image)
        product_id = match.id.rsplit("-", 1)[0]

        # Simpan metadata hanya sekali
        if not combined_scores[product_id]["metadata"]:
            combined_scores[product_id]["metadata"] = match.metadata

        # Simpan skor sesuai modalitas
        combined_scores[product_id][f"{vector_type}_score"] = match.score


def search_multimodal(
    query_vector_text,
    query_vector_image,
    text_weight: float = 1.0,
    image_weight: float = 1.0,
    min_score: float = 0.5,
    top_k: int = TOP_K
):
     # Dictionary agregasi skor text & image per produk
    combined_scores = defaultdict(lambda: {
        "text_score": 0.0,
        "image_score": 0.0,
        "metadata": None
    })

    
    # --- Query berbasis teks ---
    if query_vector_text:
        text_results = index.query(
            vector=query_vector_text,
            top_k=top_k,
            include_metadata=True,
            filter={"vector_type": "text"}
        )
        process_pinecone_results(
            text_results, combined_scores, "text", min_score
        )

    # --- Query berbasis gambar ---
    if query_vector_image:
        image_results = index.query(
            vector=query_vector_image,
            top_k=top_k,
            include_metadata=True,
            filter={"vector_type": "image"}
        )
        process_pinecone_results(
            image_results, combined_scores, "image", min_score
        )

    # --- Reranking (dynamic weight per product) ---
    results = []

    for product_id, data in combined_scores.items():
        score_sum = 0.0
        weight_sum = 0.0

        if query_vector_text is not None and data["text_score"] > 0:
            score_sum += data["text_score"] * text_weight
            weight_sum += text_weight

        if query_vector_image is not None and data["image_score"] > 0:
            score_sum += data["image_score"] * image_weight
            weight_sum += image_weight

        # Tidak ada kontribusi sama sekali
        if weight_sum == 0:
            continue

        combined_score = score_sum / weight_sum

        if combined_score >= min_score:
            results.append({
                "id": product_id,
                "score": combined_score,
                "metadata": data["metadata"]
            })

    # --- Sort & Top-K ---
    return sorted(
        results,
        key=operator.itemgetter("score"),
        reverse=True
    )[:top_k]
    # return combined_scores

In [ ]:
final_top_k=search_multimodal(query_vector_text, query_vector_image, min_score=0.5)

defaultdict(<function __main__.search_multimodal.<locals>.<lambda>()>,
            {'prod_007': {'text_score': 0.854410291,
              'image_score': 0.0,
              'metadata': {'category': 'Furniture',
               'description': 'Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.',
               'image_path': 'gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg',
               'name': 'Kursi Santai Minimalis Kontemporer',
               'price': 250000.0,
               'product_id': 'prod_007',
               'source': 'multi-modal-product-catalog',
               'vector_type': 'text'}},
             'prod_003': {'text_score': 0.80772984,
              'image_score': 0.0,
              'metadata': {'category': 'Furniture',
   

In [132]:
content_blocks

[{'type': 'image',
  'url': 'https://storage.googleapis.com/cloud-training/OCBL447/gemini-app/images/living_room.jpeg'},
 {'type': 'text',
  'text': 'Berikan saya rekomendasi furniture untuk ruang tamu berikut.'}]

In [133]:
# Tes 1:uji LLM untuk rekomendasi produk berdasarkan konteks pencarian
context_str = final_top_k
query_text = content_blocks[0].get('text')
final_prompt_template = f"""
Anda adalah asisten rekomendasi produk yang ramah. Jawab pertanyaan pengguna berdasarkan pertanyaan pengguna dan input pengguna dengan ketentuan di bawah ini.
1. Konfirmasi bahwa Anda menemukan produk serupa.
2. Berikan rekomendasi 1-3 produk yang paling relevan, produk harus sesuai dengan kebutuhan pengguna, kalau pengguna butuh meja saja jangan kasih rekomendasi selain meja.
3. Selalu pertahankan nada profesional dan bantu.
5. Output harus format JSON, key-nya : 'name', 'price', 'description', 'image_url'.

PERTANYAAN PENGGUNA: {query_text}
INPUT PENGGUNA: {query}
Hasil Pencarian Produk yang Mungkin Serupa:
{context_str}
"""
answer = llm.invoke(final_prompt_template).content
Markdown((answer))

Tentu, saya menemukan beberapa produk yang serupa berdasarkan deskripsi yang Anda berikan.

Berikut adalah rekomendasi produk yang mungkin sesuai dengan kebutuhan Anda untuk armchair/kursi santai bergaya minimalis:

```json
[
  {
    "name": "Kursi Santai Minimalis Kontemporer",
    "price": 250000.0,
    "description": "Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.",
    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg"
  },
  {
    "name": "Kursi Santai Minimalis",
    "price": 100000.0,
    "description": "Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.",
    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg"
  }
]
```

In [ ]:
# Tes 2: uji dengan meminta rekomendasi produk yang tidak ada
context_str = final_top_k
query_text = 'Apakah ada produk pisau?'
final_prompt_template = f"""
Anda adalah asisten rekomendasi produk yang ramah. Jawab pertanyaan pengguna berdasarkan pertanyaan pengguna dan input pengguna dengan ketentuan di bawah ini.
1. Berikan rekomendasi 1-3 produk yang paling relevan, produk harus sesuai dengan kebutuhan pengguna, kalau pengguna butuh meja saja jangan kasih rekomendasi selain meja.
2. Output harus format JSON, tidak boleh ada teks lain, key-nya : 'name', 'price', 'description', 'image_url'.


PERTANYAAN PENGGUNA: {query_text}
INPUT PENGGUNA: {query}
Hasil Pencarian Produk yang Mungkin Serupa:
{context_str}
"""
answer = llm.invoke(final_prompt_template)
Markdown((answer.content))

[]

In [144]:
# Tes 3: uji LLM untuk rekomendasi produk berdasarkan konteks pencarian (output json))
context_str = final_top_k
query_text = content_blocks[0].get('text')
final_prompt_template = f"""
Anda adalah asisten rekomendasi produk yang ramah. Jawab pertanyaan pengguna berdasarkan pertanyaan pengguna dan input pengguna dengan ketentuan di bawah ini.
1. Berikan rekomendasi 1-3 produk yang paling relevan, produk harus sesuai dengan kebutuhan pengguna, kalau pengguna butuh meja saja jangan kasih rekomendasi selain meja.
2. Output harus format JSON, tidak boleh ada teks lain, key-nya : 'name', 'price', 'description', 'image_url'.


PERTANYAAN PENGGUNA: {query_text}
INPUT PENGGUNA: {query}
Hasil Pencarian Produk yang Mungkin Serupa:
{context_str}
"""
answer = llm.invoke(final_prompt_template)
Markdown((answer.content))

```json
[
  {
    "name": "Kursi Santai Minimalis Kontemporer",
    "price": 250000.0,
    "description": "Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.",
    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg"
  },
  {
    "name": "Kursi Santai Minimalis",
    "price": 100000.0,
    "description": "Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.",
    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg"
  }
]
```

In [145]:
answer

AIMessage(content='```json\n[\n  {\n    "name": "Kursi Santai Minimalis Kontemporer",\n    "price": 250000.0,\n    "description": "Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.",\n    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg"\n  },\n  {\n    "name": "Kursi Santai Minimalis",\n    "price": 100000.0,\n    "description": "Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.",\n    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg"\n  }\n]\n```', additional_kwargs={}, response_metadata={'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': [], 'model_provid

In [146]:
answer.text.strip()

'```json\n[\n  {\n    "name": "Kursi Santai Minimalis Kontemporer",\n    "price": 250000.0,\n    "description": "Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.",\n    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg"\n  },\n  {\n    "name": "Kursi Santai Minimalis",\n    "price": 100000.0,\n    "description": "Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.",\n    "image_url": "gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg"\n  }\n]\n```'

In [151]:
raw_answer= answer.content
raw_answer = raw_answer.replace("```json", "").replace("```", "").strip()

try:
    result = json.loads(raw_answer)
    print(result)
except Exception as e:
    print("Gagal parse JSON:", e)
    print("Raw output:", raw)

[{'name': 'Kursi Santai Minimalis Kontemporer', 'price': 250000.0, 'description': 'Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.', 'image_url': 'gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg'}, {'name': 'Kursi Santai Minimalis', 'price': 100000.0, 'description': 'Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.', 'image_url': 'gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg'}]


In [155]:
# membuat susunan markdown 
fmt=[]
for item in result:
    name = item.get('name', 'N/A')
    price = item.get('price', 'N/A')
    description = item.get('description', 'N/A')
    image_url = item.get('image_url', 'N/A')
    fmt.append(f"Nama Produk: {name}\nHarga: {price}\nDeskripsi: {description}\nImage URL: {image_url}")

Markdown(str("\n\n".join(fmt)))

Nama Produk: Kursi Santai Minimalis Kontemporer
Harga: 250000.0
Deskripsi: Kursi santai tunggal bergaya Minimalis Kontemporer yang cocok diletakkan di sudut baca atau sebagai kursi aksen di ruang tamu, terbuat dari Material Pelapis Kain Tenun berwarna Abu-abu Muda yang nyaman, ditopang oleh Kaki Kayu Solid Alami yang tirus, memberikan sentuhan kehangatan dan elegan.
Image URL: gs://latihan-my-ecommerce-images1/images/kursi_santai.jpg

Nama Produk: Kursi Santai Minimalis
Harga: 100000.0
Deskripsi: Kursi santai dengan desain minimalis, bahan kayu solid dan dudukan nyaman, cocok untuk ruang tamu atau balkon, tahan lama dan mudah dipadukan dengan dekorasi rumah modern.
Image URL: gs://latihan-my-ecommerce-images1/images/kursi_kayu.jpg